In [1]:
import polars as pl
import polars.selectors as cs

# Aggregating rates across rows and columns

## Example - The mean of a rate isn't the combined rate

In [74]:
(df :=
 pl.DataFrame({'Group': 3*['A'] + 2*['B'] + 3*['C'],
               'Value': [1, 2, 3, 5, 2, 3, 3, 2]})
)

Group,Value
str,i64
"""A""",1
"""A""",2
"""A""",3
"""B""",5
"""B""",2
"""C""",3
"""C""",3
"""C""",2


#### Group means

In [75]:
(df
.group_by('Group')
.agg(mean_value = pl.col('Value').mean())
)

Group,mean_value
str,f64
"""A""",2.0
"""B""",3.5
"""C""",2.666667


#### Mean of groups means

In [76]:
(mean_of_group_means :=
 df
.group_by('Group')
.agg(mean_value = pl.col('Value').mean())
.select(pl.col('mean_value').mean().alias('mean_of_means'))
)

mean_of_means
f64
2.722222


#### Grand mean

In [77]:
(grand_mean :=
 df
.select(pl.col('Value').mean().alias('grand_mean'))
)

grand_mean
f64
2.625


#### Mean of the group means `!=` grand mean

In [78]:
mean_of_group_means - grand_mean

mean_of_means
f64
0.097222


### Examples of common rates

1. Mean,
2. Percent of total and other ratios,
3. Population rates (e.g., X per 1K people),
4. Anything you describe using "per"


## Review - Aggregating rates

When aggregating a rate across groups, we need to

1. Aggregate the numerator and demon, then
2. Compute the rate from to using the aggregated values.

In [79]:
(grand_mean_via_groups :=
 df
.group_by('Group')
.agg(sum_value = pl.col('Value').sum(),
     cnt_value = pl.col('Value').count())
.select(pl.col('sum_value').sum().alias('sum_of_sums'),
        pl.col('cnt_value').sum().alias('cnt_of_cnts'),
       )
.with_columns(grand_mean = pl.col('sum_of_sums')/pl.col('cnt_of_cnts'))
.drop('sum_of_sums', 'cnt_of_cnts')
)

grand_mean
f64
2.625


In [27]:
grand_mean_via_groups - grand_mean

grand_mean
f64
0.0


### Why do we care?

Because sometimes the data is already an aggregated value!

In [80]:
( auto_sales :=
  pl.read_csv('./data/auto_sales_*.csv')
  .rename({'':'ID'})
)

ID,Salesperson,Compact,Sedan,SUV,Truck
i64,str,i64,i64,i64,i64
0,"""Ann""",22,18,15,12
1,"""Bob""",19,12,17,20
2,"""Yolanda""",19,8,32,15
3,"""Xerxes""",12,23,18,9
0,"""Ann""",22,18,15,12
1,"""Bob""",20,14,6,24
2,"""Yolanda""",19,10,28,17
3,"""Xerxes""",11,27,17,9


## Aggregating a rate across rows and columns

1. Compute the numerator and denominator row aggregations using `group_by` and `agg`.
2. Compute the numerator and denominator column aggregations using `reduce` or `fold`.

In [81]:
(type_columns := 
 [c for c in auto_sales.columns if c not in ('ID','Salesperson')]
)

['Compact', 'Sedan', 'SUV', 'Truck']

In [83]:
(auto_sales
 .group_by('Salesperson')
 .agg(**{f'sum_{c}':pl.col(c).sum() for c in type_columns},
      **{f'cnt_{c}':pl.col(c).count() for c in type_columns},
     )
 .with_columns(sum_types = pl.reduce(lambda acc, col: acc + col, cs.starts_with('sum')),
               cnt_types = pl.reduce(lambda acc, col: acc + col, cs.starts_with('cnt')),
              )
 .with_columns(grand_mean = pl.col('sum_types')/pl.col('cnt_types'))
#  .drop(cs.starts_with('sum') | cs.starts_with('cnt'))
 
)

Salesperson,sum_Compact,sum_Sedan,sum_SUV,sum_Truck,cnt_Compact,cnt_Sedan,cnt_SUV,cnt_Truck,sum_types,cnt_types,grand_mean
str,i64,i64,i64,i64,u32,u32,u32,u32,i64,u32,f64
"""Yolanda""",38,18,60,32,2,2,2,2,148,8,18.5
"""Bob""",39,26,23,44,2,2,2,2,132,8,16.5
"""Ann""",44,36,30,24,2,2,2,2,134,8,16.75
"""Xerxes""",23,50,35,18,2,2,2,2,126,8,15.75


## <font color="red"> Exercise 3.9 </font> - World Bank Population Ratio (urban/total) for each region over the 1980's

Use the approach illustrated in the last example to compute the answers to the following questions:

1. **Question 1**.  What is the overall ratio of Urban to Total population for each region across all years in the 1980's?
2. **Question 2**.  For each region, compute the probability that a randomly selected person from that region in the 1980's was from an urban area? [Compute this using Bayes rule, e.g. $\frac{successes + 1}{trials + 2}$]

Do this by

1. Loading the raw WB data,
2. Select the columns of interest using column selectors,
3. Filter to the two measures of interest,
4. Unstack the two measures,
5. Group and aggregate the numerator and denominator of the ratio across rows,
6. Use `reduce` or `fold` to aggregate the numerator and denominator of the ratio across columns, and
7. Compute the ratio.

Next, redo the query, but this time replace calls to `reduce`/`fold` with a "horizontal" helper function.

In [1]:
# your fold/reduce code here

In [2]:
# your refactored code here